In [1]:
import os
if os.path.isdir('../../UnifiedOntology-Pipeline'): os.chdir('../')

In [2]:
import import_ipynb
from tool.extractClass import *
from pipeline.connectIndividuals import getNamespace, clear_serialize

/home/s2416084/miniconda3/envs/bert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
loginHuggingFace()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `colab` has been saved to /home/s2416084/.cache/huggingface/stored_tokens
Your token has been saved to /home/s2416084/.cache/huggingface/token
Login successful.
The current active token is: `colab`


In [4]:
path_to_onto = "./pipeline/3_mergeOutput/UnifiedOntology.ttl"
df = getOntoClasses(path_to_onto)

In [5]:
df.shape

(48, 1)

In [6]:
name_tokenizer = "bert-base-uncased"
name_model = "OhWayTee/bert-taxonomy2"
name_pipeline = "text-classification"

In [7]:
classifier = getClassifier(name_tokenizer, name_model, name_pipeline)

Device set to use cuda:0


In [41]:
print(classifier("APIKey [SEP] Company"))

[{'label': 'LABEL_2', 'score': 0.9957928657531738}]


In [9]:
df_pred = getPredictions(classifier, df)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [10]:
df_top = sortPredictions(df_pred)

In [11]:
df_top.shape

(108, 4)

In [12]:
confidence_threshold = 0.9

In [13]:
df_irrelevant= df_top[(df_top['label'] == "LABEL_0") ] # 
df_superclass = df_top[(df_top['label'] == "LABEL_1") & (df_top['score'] >= confidence_threshold)] # classB is SUBCLASS of classA
df_subclass = df_top[(df_top['label'] == "LABEL_2") & (df_top['score'] >= confidence_threshold)] # classA is SUBCLASS of classB
df_sameclass = df_top[(df_top['label'] == "LABEL_3") & (df_top['score'] >= confidence_threshold)] # classA is SAME_CLASS with classB

In [22]:
[df_superclass.shape, df_subclass.shape, df_sameclass.shape]

[(17, 4), (40, 4), (47, 4)]

In [23]:
df_superclass_filtered = df_superclass.sort_values('score', ascending=False).drop_duplicates('classB') # remove duplicate subclasses
df_subclass_filtered = df_subclass.sort_values('score', ascending=False).drop_duplicates('classA')
[df_superclass_filtered.shape, df_subclass_filtered.shape]

[(12, 4), (40, 4)]

In [28]:
g = Graph()
g.parse(path_to_onto, format='turtle')
prefix, uri = getNamespace(g)
print(prefix, uri)

ex http://example.org/ontology#


In [29]:
for idx, row in df_superclass_filtered.iterrows():
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX {prefix}: <{uri}>
    INSERT{{
        ?subclass rdfs:subClassOf ?superclass .
    }}
    WHERE {{
            {{
                ?subclass rdfs:label "{row['classB']}" .
                ?superclass rdfs:label "{row['classA']}" .
            }}
            UNION
  			{{
                ?subclass a owl:Class .
				?superclass a owl:Class .
        		FILTER(STRAFTER(STR(?subclass), "#") = "{row['classB']}") 
        		FILTER(STRAFTER(STR(?superclass), "#") = "{row['classA']}") 
  			}}
        }}
    """
    g.update(query)

In [30]:
for idx, row in df_subclass_filtered.iterrows():
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX {prefix}: <{uri}>
    INSERT{{
        ?subclass rdfs:subClassOf ?superclass .
    }}
    WHERE {{
            {{
                ?subclass rdfs:label "{row['classA']}" .
                ?superclass rdfs:label "{row['classB']}" .
            }}
            UNION
  			{{
                ?subclass a owl:Class .
				?superclass a owl:Class .
        		FILTER(STRAFTER(STR(?subclass), "#") = "{row['classA']}") 
        		FILTER(STRAFTER(STR(?superclass), "#") = "{row['classB']}") 
  			}}
        }}
    """
    g.update(query)

In [31]:
output_path = "./refine/4_refineOutput/RefinedUnifiedOntology.ttl"

clear_serialize(g, output_path)